In [14]:
'''
read file full of ciphertexts
read in all 997 power traces
implement keeloq decryption but with printing out a specific round
use numpy to perform correlation with power trace
'''
import numpy
import numpy.linalg as la
import datetime
import struct
import csv
from time import time
import matplotlib.pyplot as plt
import random
import math
from operator import itemgetter


path="$PATH"
KeeLoq_NLF=0x3A5C742E
 
def bit(x,n):
    ''' Get the n'th bit from x'''
    y=(((x)>>(n))&1)
    return y
    
def g5(x,a,b,c,d,e):
    ''' Non linear mixing of bits'''
    y=(bit(x,a)+bit(x,b)*2+bit(x,c)*4+bit(x,d)*8+bit(x,e)*16)
    return y


        
def read_cipher():
        ciphertext=[]
        ciphertextfile=open(r"./25C3_HCS301/ciphertexts.txt","r")
        for line in ciphertextfile:
            ciphertext.append(int(line,16))
        #print(ciphertext[888])
        #line 889 index starts at 0
        ciphertextfile.close()
        return ciphertext
        
def read_powertrace():
    Peak=[]
    for i in range(997):
        Peak.append(i)
        Peak[i]=[]
        with open(f"./25C3_HCS301/Peak{i+1}.dat",mode='rb') as file:
            for j in range(1650):
                byte = file.read(8)
                Peak[i].append(struct.unpack('d',byte)[0])
        file.close()
    return Peak

ciphers = read_cipher()
power_traces = read_powertrace()


numpy_peak=numpy.array(power_traces)
transpose_peak=numpy.transpose(numpy_peak)
numpy_cipher=numpy.array(ciphertext)


In [1]:
#The key starts at bit 15 and then gets left shifted (next bit used is 14)

def decrypt_round(cipher,device_key)
    return ((cipher<<1) ^bit(cipher,31) ^bit(cipher,15) ^bit(device_key,(15-r)&63) ^bit(KeeLoq_NLF,g5(cipher,0,8,19,25,30))) & 0xFFFFFFFF
        
def partial_decrypt (cipher,device_key,stop):

    for r in range(stop):
        cipher = decrypt_round(cipher,device_key)
    y0=cipher&0xFFFFFFFF
    return y0

def full_decrypt(cipher,device_key):
    return partial_decrypt (cipher,device_key,528)

def partial_decrypt_power (cipher,device_key,stop):
    
    for r in range(stop):
        cipher = decrypt_round(cipher,device_key)
        if r == stop - 2 : y1 = cipher&0xFFFFFFFF
        if r == stop - 1 : y0 = cipher&0xFFFFFFFF
        
    power = bin(y1^y0).count('1')

    return power
        
def partial_decrypt_power_array (cipher,device_key,stop):
        power=[]
        prev=cipher
        for r in range(stop):
            cipher = decrypt_round(cipher,device_key)
            curr=cipher
            power.append((bin(prev^curr)).count('1')) #hamming distance between this and previous round
            prev=cipher
        power=power[::-1]#reverse the list so that the most recent decryption is first
        
        return power        

def partial_decrypt_power_array_with_noise (cipher,device_key,stop):
        cipher_array=[]
        power=[]
        cipher_array.append(cipher) #add the initial cipher
        for r in range(2*stop):
            cipher = decrypt_round(cipher,device_key)
            cipher_array.append(cipher) #this will have length 2*stop+1 
            
        for r in range(len(cipher_array)-1): #this will have length 2*stop
            phyp=bin(cipher_array[r]^[r+1]).count('1')#theoretical power hypothesis
            power.append(random.gauss(phyp,0.1*phyp)) #hamming distance between this and previous round
            #random guassian noise is added to simulate reality, although we dont know what kind of noise exists in reality, most likely variance it is proportional to peak power 
          
        return power
        
def partial_decrypt_array (cipher,device_key,stop):
        cipher_array=[]
        power=[]
        cipher_array.append(cipher) #add the initial cipher
        
        for r in range(2*stop):
            cipher = decrypt_round(cipher,device_key)
            cipher_array.append(cipher) # this will have length 2*stop+1 
        
        for r in range(len(cipher_array) - 1):#this will have length 2*stop
            p_hyp = bin(cipher_array[r]^[r+1]).count('1') #theoretical power hypothesis
            power.append(p_hyp) #hamming distance between this and previous round
            #random guassian noise is added to simulate raelity, although we dont know what kind of noise exists in reality, most likely variance it is proportional to peak power 
          
        return power

def partial_decrypt_numpy_power (cipher,device_key,stop):
    
    #print("key "+str(r)+": "+str(hex(device_key))+": "+str(bin(device_key)[2:]).zfill(64))
    for r in range(stop):
        cipher = decrypt_round(cipher,device_key)
        r=r+1
        if r==stop - 2: y1=cipher&0xFFFFFFFF
        if r==stop - 1: y0=cipher&0xFFFFFFFF              
    
    power=(y1^y0)
    
    return(power)  

        
def key_register(key):
    #print(key) #take a base 10 digit convert it into the actually key in the right format (flipped and left rotated 16 bits)
    key=(bin(key))[2:]#get rid of 0b
    #print(key)
    key=key.zfill(64)#64 chars
    #print(key)
    key=key[::-1]#reverse
    #print(key)
    key=int(key,2)#convert to binary
    #print(key)
    key=(((key>>(64-16))^(key<<(16)))&0xFFFFFFFFFFFF)#rotate 48 bits to right or 16 bits to left
    #print(key)
    return key
  
def bin_rep(int):
    return(bin(int)[2:].zfill(16))
  
def algorithm1(keys,numpy_cipher,dummy_bit_guess,master_power):
    dummy_power=[]#stores the power for a key guess
    dummy_corr=[]#stores the correlation between the master key power and a key guesses power
    max_dummy_corr=[]
    bin_key=[]#bin key is a string of the binary representation of the key
    avg_dummy_corr=[]  #largely useless  
    print(keys)
    
    for key in keys:
    bin_key.append(bin(key)[2:].zfill(64)[(64-dummy_bit_guess):]) 
    dummy_power.append([])#For each key guess we need to get the hypothetical power so append an array that will contain the power for each cipher
    dummy_corr.append([])#correlate the above with the master key
    key_guess_power=[]#contains the power for a particular key guess
    
    for cipher in range(len(numpy_cipher)): #for each cipher perform a partial decrypt dummy_bit_guesses deep
        key_guess_power_cipher_array=[]
        key_guess_power_cipher_array=partial_decrypt_array(numpy_cipher[cipher],keys[key],dummy_bit_guess)#array length bit_guess for one cipher 1d
        key_guess_power.append(key_guess_power_cipher_array)#append it to an array for each cipher, array is cipher*bit_guess_round 2d
    key_guess_power=numpy.transpose(key_guess_power)
    dummy_power[key].append(key_guess_power)#transpose so it is now bit_guess_round*cipher and append it to the dummy_power array

    for dummy_bit in range(len(dummy_power[key][0])):
        dummy_corr[key].append(numpy.corrcoef(master_power[dummy_bit_guess],dummy_power[key][0][dummy_bit],1,1)[0,1]) #for each key guess correlate it to the master key guess    
    
    max_dummy_corr.append((max(dummy_corr[key]),key,bin_key[key]))
    print(max_dummy_corr[key])
    likely_keys=sorted(max_dummy_corr, key=itemgetter(0))[:-1*(return_keys+1):-1]  
    return(likely_keys,dummy_corr) #Get the four most likely(4 highest max correlation) key guesses, this is the output
 

ciphertext=[]

for i in range(30):
    ciphertext.append(i)
numpy_cipher=numpy.array(ciphertext)

#print (numpy_cipher[0])

dummy_key="B427983C9A60C0F2"#random key
#dummy_key=key_register(int(dummy_key,16))

print("dummy master key:   "+bin(int(dummy_key,16))[2:].zfill(64)[48:])
master_power=[]

for cipher in range(len(numpy_cipher)):
    master_power.append(partial_decrypt_power_array_with_noise(numpy_cipher[cipher],int(dummy_key,16),70))


master_power=numpy.transpose(master_power)
#numpy.savetxt(path+"dummy DPA master.csv", master_power, delimiter=",", fmt="%s")
    
#dummy_key="0000000000000000"#null key

#make the reverse of this function
#and then test using the canonical C code

time1 = time()

round=0 #this will increment after guessing a while batch of keys
bit_guess=8   #how many bits of the key we want to guess
key_guesses = 2**bit_guess #number of key guesses
return_key_bit_guess=2
return_keys=2**return_key_bit_guess #return this most probable keys
round_bit_guess=4 #guess a constant amount of keys per round
  
  

In [1]:
#turn this monster into a function that takes m key guesses and returns the n most probable key guesses with k known bits  

likely_keys=[] #dictionary( of sorts) of the max correlation, the index and a string of the key
for round in range(int(math.ceil((64-bit_guess)/round_bit_guess))+1):
#for round in range(6):
    dummy_bit_guess=bit_guess+round*round_bit_guess #This is the depth of the bit we are guessing
    print (round,"",dummy_bit_guess)
    if round==0:
        key_g=[]
        key_g = range(key_guesses)#inital keys is just the first key_guesses # in an array, this needs to be modified after the first round, or needs some clever code to handle the fact when k=0 bits
    else:
        key_g=[]
        for likely_key in range(len(likely_keys)):
            for key in range(2**round_bit_guess):
                key_g.append(((key<<(dummy_bit_guess-round_bit_guess))^int(likely_keys[likely_key][2],2)))
                #key_g[key]=((key<<(dummy_bit_guess-round_bit_guess))^int(likely_keys[likely_key][2],2))
                #print(bin_rep(key<<(dummy_bit_guess-round_bit_guess)),bin_rep(int(likely_keys[likely_key][2],2)))
                #print(bin_rep(key_g[key]),key_g[key])
                #print(key_g[key])

        
        
  #print(likely_keys)
  #print(key_g)
    output=algorithm1(key_g,numpy_cipher,dummy_bit_guess,master_power)
    corr=output[1]
    likely_keys=output[0]

    plt.close()
    colormap = plt.cm.gist_ncar
    plt.gca().set_color_cycle([colormap(i) for i in numpy.linspace(0, 0.9, key_guesses)])
    fig = plt.gcf()
    fig.set_size_inches(19.20,10.80)
    for key in range(len(keys)):
        plt.plot(range(len(corr[key])),corr[key])
    
    #plt.legend(bin_key, loc='best')
    plt.savefig(path+"round "+str(round)+" - "+str(dummy_bit_guess)+" bit guesses traces DPA plots.png",dpi=400)
    #print(path+"round "+str(round)+" - "+str(dummy_bit_guess)+" bit guesses traces DPA plots.png")

time2 = time() 
print("processed stuff:", time2 - time1, "s")

time6 = time()
print("Time taken to make picture ",dummy_bit_guess,"key guesses: ", time6 - time2, "s")

In [1]:
#myfile = open(path+"DPA.csv", 'wb')
#wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#wr.writerow(correlation_array)
#myfile.close()

dummy_power=numpy.transpose(dummy_power,(1,0,2,3))#swap first and second dimensions/axes 2nd axes is one element containing a 2d array
print(dummy_power.shape)
numpy.savetxt(path+"dummy DPA corr.csv", dummy_corr, delimiter=",")
numpy.savetxt(path+"dummy DPA key.csv", keys, delimiter=",")
numpy.savetxt(path+"dummy DPA power.csv", dummy_power[0], delimiter=",",fmt="%s")
numpy.savetxt(path+"dummy DPA master.csv", master_power, delimiter=",", fmt="%s")
#numpy.savetxt(path+"dummy DPA partial cipher.csv", cipher_partial, delimiter=",")
#numpy.savetxt(path+"dummy DPA max correlation.csv", max_corr, delimiter=",")

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(ciphertext[888])? (2478799849.py, line 147)